In [1]:
import pandas as pd

df = pd.read_csv('../dados/acordaos-selecionada.csv', sep = '|')
df.head()

,acordao,arquivo,areas,texto
0,297/2016-P,547240.txt,Responsabilidade,TRIBUNAL DE CONTAS DA UNIÃO\tTC 010.084/2015-0...
1,366/2016-P,549518.txt,Finanças Públicas,TRIBUNAL DE CONTAS DA UNIÃO\tTC 005.933/2014-5...
2,944/2016-P,554399.txt,Responsabilidade,TRIBUNAL DE CONTAS DA UNIÃO\tTC 042.038/2012-0...
3,30/2016-P,545010.txt,Direito Processual,TRIBUNAL DE CONTAS DA UNIÃO\tTC 000.742/2014-7...
4,55/2016-P,544046.txt,Pessoal,;-;;Wania Lucia Pasquarelli do NascimentoTCUWa...


In [2]:
df.shape

(10524, 4)

In [3]:
areas = df.groupby(['areas']).groups.keys()
areas

dict_keys(['Competência do TCU', 'Contrato Administrativo', 'Contrato Administrativo,Competência do TCU', 'Contrato Administrativo,Desestatização', 'Contrato Administrativo,Finanças Públicas', 'Contrato Administrativo,Pessoal', 'Convênio', 'Convênio,Competência do TCU', 'Convênio,Contrato Administrativo', 'Convênio,Contrato Administrativo,Finanças Públicas', 'Convênio,Direito Processual', 'Convênio,Finanças Públicas', 'Convênio,Gestão Administrativa', 'Convênio,Gestão Administrativa,Competência do TCU', 'Convênio,Gestão Administrativa,Contrato Administrativo', 'Convênio,Gestão Administrativa,Licitação', 'Convênio,Gestão Administrativa,Pessoal', 'Convênio,Licitação', 'Convênio,Licitação,Desestatização,Finanças Públicas', 'Convênio,Licitação,Pessoal,Finanças Públicas', 'Convênio,Pessoal', 'Convênio,Responsabilidade', 'Convênio,Responsabilidade,Competência do TCU', 'Convênio,Responsabilidade,Gestão Administrativa', 'Desestatização', 'Desestatização,Competência do TCU', 'Desestatização,Fin

In [4]:
from sklearn.preprocessing import LabelBinarizer

lbArea = LabelBinarizer()
lbArea.fit([x for x in areas])
lbArea.classes_

array(['Competência do TCU', 'Contrato Administrativo',
       'Contrato Administrativo,Competência do TCU',
       'Contrato Administrativo,Desestatização',
       'Contrato Administrativo,Finanças Públicas',
       'Contrato Administrativo,Pessoal', 'Convênio',
       'Convênio,Competência do TCU', 'Convênio,Contrato Administrativo',
       'Convênio,Contrato Administrativo,Finanças Públicas',
       'Convênio,Direito Processual', 'Convênio,Finanças Públicas',
       'Convênio,Gestão Administrativa',
       'Convênio,Gestão Administrativa,Competência do TCU',
       'Convênio,Gestão Administrativa,Contrato Administrativo',
       'Convênio,Gestão Administrativa,Licitação',
       'Convênio,Gestão Administrativa,Pessoal', 'Convênio,Licitação',
       'Convênio,Licitação,Desestatização,Finanças Públicas',
       'Convênio,Licitação,Pessoal,Finanças Públicas', 'Convênio,Pessoal',
       'Convênio,Responsabilidade',
       'Convênio,Responsabilidade,Competência do TCU',
       'Convênio,

In [5]:
y = lbArea.transform(df['areas'])
y.shape

(10524, 98)

In [6]:
from keras.preprocessing.text import Tokenizer
import numpy as np

vocabulario = 350000
limite_texto = 40000
dim_vetor = 50

tokenizer = Tokenizer(num_words=vocabulario)
tokenizer.fit_on_texts(df['texto'])

sequences = tokenizer.texts_to_sequences(df['texto'])

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Using TensorFlow backend.


Found 344296 unique tokens.


In [7]:
from keras.preprocessing.sequence import pad_sequences

x = pad_sequences(sequences, maxlen=limite_texto)

print('Shape of data tensor:', x.shape)

Shape of data tensor: (10524, 40000)


In [8]:
from gensim.models import Word2Vec

model = Word2Vec.load('../vocabularios/modelo-acordaos-50.w2v')

/home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [9]:
# create a weight matrix for words in training docs

embedding_matrix = np.zeros((vocabulario, dim_vetor))

ok = 0
for word, i in tokenizer.word_index.items():
    if word in model:
        embedding_matrix[i] = model[word]
        ok += 1
print('Vocabulario:', i)
print('Encontrados no modelo:', ok, '=', ok * 100. / i)

/home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  import sys
/home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


Vocabulario: 344296
Encontrados no modelo: 90722 = 26.350001161791017


In [10]:
from keras.layers import Embedding

embedding = Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=False)

W0226 15:12:03.759488 140648603031360 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.



In [11]:
x.shape, y.shape

((10524, 40000), (10524, 98))

# Treinamento

## Embedding com pesos fixos

In [12]:
from keras.models import Sequential
from keras.layers import Flatten, Dense
from keras.layers.core import Dropout

model = Sequential()
model.add(embedding)
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adadelta',  metrics=["categorical_accuracy"])
model.summary()
history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

W0226 15:12:03.994921 140648603031360 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0226 15:12:03.996780 140648603031360 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0226 15:12:04.004015 140648603031360 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0226 15:12:04.004564 140648603031360 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:181: The name tf.ConfigProto is deprecate

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 40000, 50)         17500000  
_________________________________________________________________
flatten_1 (Flatten)          (None, 2000000)           0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                128000064 
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               16640     
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 98)                25186     
Total para